<a href="https://colab.research.google.com/github/SimonielMusyoki/Data-Science/blob/master/AppliancesScrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
from glob import glob
from bs4 import BeautifulSoup
from datetime import datetime
from time import sleep

In [ ]:
HEADERS = ({'User-Agent':
                'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

In [15]:
def get_product_data(soup_obj) -> dict:
    try:
        product_title = soup_obj.find('div', {'class': 'product-title'}).find('h1').string
        product_brand = soup_obj.find('p', {'class': 'product-brand'}).find('a').string
        product_price = soup_obj.find('span', {'class': 'stockrecord-price-current'}).string.strip().replace("KES ",
                                                                                                             "").replace(
            ",", "")
        dict_ = {
            "name": product_title,
            "brand": product_brand,
            "price": int(product_price)
        }

        return dict_
    except:
        pass


In [ ]:
def get_soup_from_url(url) -> BeautifulSoup:
    product_page = requests.get(url, headers=HEADERS)
    product_soup = BeautifulSoup(product_page.content, features="lxml")
    return product_soup

In [ ]:
def extract_urls_from_page(product_list_url: str) -> list:
    links = list()
    soup = get_soup_from_url(product_list_url)
    anchor_tags = soup.find_all('div', {'class': 'product-item'})
    for anchor_tag in anchor_tags:
        links.append(anchor_tag.find('a')['href'])
    return links

In [ ]:
def extract_pages_count(product_list_url: str) -> int:
    soup = get_soup_from_url(product_list_url)
    pages_el = soup.find_all("li", {'class': "page-paginator-item"})[-2].text.split(" ")[0]
    return int(pages_el)

In [23]:
def fetch_data(links: list) -> list:
  print(links)
  data = list()
  for link in links:
      pages = extract_pages_count(f'https://hotpoint.co.ke/catalogue/category/{link}')
      for i in range(1, pages):
          urls = extract_urls_from_page(f"https://hotpoint.co.ke/catalogue/category/{link}?page={i}")
          for url in urls:
            try:
              pdp = f'https://hotpoint.co.ke{url}'
              pdp_soup = get_soup_from_url(pdp)
              product_dict = get_product_data(pdp_soup)
              
              product_dict['url'] = f'https://hotpoint.co.ke{url}'
              data.append(product_dict)
              print(product_dict)
            except:
              pass
  return data

In [24]:
def write_to_excel(results) -> None:
  df = pd.DataFrame.from_dict(results)
  df.to_excel('hotpoint.xlsx')

In [25]:
hotpoint_links = ['phones-tablets/','camera-optics/','fridges-freezers/','washers-dryers/','cookers-ovens/','small-appliances/','small-home-appliances/','health-personal-care/','built-in-appliances/','commercial/']
scraped_data = fetch_data(hotpoint_links)
write_to_excel(scraped_data)



['phones-tablets/', 'camera-optics/', 'fridges-freezers/', 'washers-dryers/', 'cookers-ovens/', 'small-appliances/', 'small-home-appliances/', 'health-personal-care/', 'built-in-appliances/', 'commercial/']
{'name': 'Apple iPhone 13 mini - 512GB', 'brand': 'Apple', 'price': 159995, 'url': 'https://hotpoint.co.ke/catalogue/apple-iphone-13-mini-512gb_3225/'}
{'name': 'Apple iPhone 12 Pro Max 256GB - Blue', 'brand': 'Apple', 'price': 159995, 'url': 'https://hotpoint.co.ke/catalogue/apple-iphone-12-pro-max-256gb-blue_3056/'}
{'name': 'Apple iPhone 12 Pro 256GB - Blue', 'brand': 'Apple', 'price': 162995, 'url': 'https://hotpoint.co.ke/catalogue/apple-iphone-12-pro-256gb-blue_3055/'}
{'name': 'Samsung S22 256GB - Phantom Black', 'brand': 'Samsung', 'price': 104000, 'url': 'https://hotpoint.co.ke/catalogue/samsung-s22-256gb-phantom-black_3463/'}
{'name': 'Belkin Wireless Charging Stand, 10W - Black', 'brand': 'Belkin', 'price': 5200, 'url': 'https://hotpoint.co.ke/catalogue/belkin-wireless-ch

IndexError: ignored

NameError: ignored